In [7]:
import os
from typing import Optional
import requests
from langchain_community.document_loaders.pdf import OnlinePDFLoader
from dotenv import load_dotenv

load_dotenv()


class ResumeService:
    def __init__(self):
        self.api_token = os.getenv("DATOCMS_API_TOKEN")
        self.resume_id = os.getenv("DATOCMS_RESUME_ID")
        self.api_url = "https://graphql.datocms.com"
        self.headers = {
            "Authorization": f"Bearer {self.api_token}",
            "Content-Type": "application/json",
        }

    async def fetch_resume_text(self) -> Optional[str]:
        """Fetch resume text"""
        try:
            # GraphQL query to fetch resume data
            query = """
            query {
                resumeUncompiled {
                    text
                }
            }
            """

            # Make GraphQL request
            response = requests.post(
                self.api_url,
                json={"query": query},
                headers=self.headers,
            )
            response.raise_for_status()

            data = response.json()

            # Check for errors in GraphQL response
            if "errors" in data:
                print(f"GraphQL errors: {data['errors']}")
                return None

            # Extract resume link from response
            # print(data)
            resume_text = data.get("data", {}).get("resumeUncompiled", {}).get("text")

            if not resume_text:

                print("No resume URL text in response")
                return None

            return resume_text

        except Exception as e:
            print(f"Error fetching resume from DatoCMS: {str(e)}")
            return None


    def format_resume_for_prompt(self, resume_text: str) -> str:
        """Format resume text for use in LLM prompt"""
        return f"""--- DATA ABOUT {{ full_name }} START ---

{resume_text}

--- DATA ABOUT {{ full_name }} END ---"""



resume_service = ResumeService()
print(await resume_service.fetch_resume_text())

#import "@preview/modern-cv:0.7.0": *

#show: resume.with(
  author: (
    firstname: "",
    lastname: "Jenslee Dsouza",
    email: "dsouzajenslee@gmail.com",
    phone: "(+91) 9146449568",
    github: "LOCKhart07",
    linkedin: "jensleedsouza", 
    homepage: "https://portfolio.lockhart.in",
    positions: (
      "Software Developer",
    ),
  ),
  date: datetime.today().display(),
  language: "en",
  colored-headers: false,
  show-footer: false,
)

= Experience

#resume-entry(
  title: "LTIMindtree",
  location: "Bengaluru & Mumbai, IN",
  date: "Sep 2022 - Present",
  description: "Software Engineer",
)
#resume-item[
  - Led the development and deployment of JOVI, an *AI-Powered Chatbot* utilizing custom LLMs and a Retrieval-Augmented Generation (RAG) pipeline, delivering highly accurate, context-aware responses. *(Java, Spring Boot, Vue.js, Python, Langchain, Sentence Transformers, Docker, Kubernetes, Websockets)*
  - Implemented a centralized *RAG File Management Portal* with a